In [11]:
from langchain_community.vectorstores import Chroma
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document
from langchain_classic.retrievers.contextual_compression import ContextualCompressionRetriever
fr
from langchain_classic.retrievers.contextual_compression import ContextualCompressionRetriever
documenom langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from dotenv import load_dotenv
import os
load_dotenv()
api_key = os.getenv("HUGGINGFACE_API_KEY")

In [12]:
# Create documents with 3 different contexts (topics)

documents = [
    # Paragraph 1: Artificial Intelligence (AI) - Technology Context
    Document(page_content="""Artificial Intelligence is transforming the modern world. 
             Famous cricketers like Sachin Tendulkar and Virat Kohli have inspired millions.
             Deep learning neural networks are being used in healthcare to diagnose diseases early.
             Renewable energy sources like solar and wind power are crucial for reducing carbon emissions.   
             However, concerns about AI safety and ethics continue to grow."""),
    
    # Paragraph 2: Cricket - Sports Context
    Document(page_content="""Cricket is one of the most popular sports in South Asia and beyond. 
             The game requires a combination of batting, bowling, and fielding skills. 
             Test matches can last up to five days, while T20 format games finish in just three hours. 
              Machine learning algorithms can now recognize patterns in data that humans might miss.
             The ICC World Cup is the most prestigious tournament in international cricket."""),
             
    # Paragraph 3: Climate Change - Environment Context
    Document(page_content="""Climate change is one of the biggest challenges facing humanity today. 
    Rising global temperatures are causing ice caps to melt and sea levels to rise. 
    Extreme weather events like hurricanes and droughts are becoming more frequent. 
    AI assistants help us with daily tasks like scheduling and information retrieval.
    Scientists warn that we must act quickly to prevent irreversible damage to our planet.""")
]

In [13]:

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


vectorstore = Chroma.from_documents(documents=documents, embedding=embedding_model, collection_name="sample_collection")



Loading weights: 100%|██████████| 103/103 [00:00<00:00, 195.96it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [14]:
llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.1-8B-Instruct",
    task="text-generation",
    huggingfacehub_api_token=api_key,
)

# model = ChatHuggingFace(llm=llm)

In [22]:
base_retriver = vectorstore.as_retriever(search_kwargs={"k": 3})
base_compressor = ChatHuggingFace(llm=llm)
compressor = LLMChainExtractor.from_llm(llm=base_compressor)

In [26]:
compression_retriver = ContextualCompressionRetriever(
    base_retriever=base_retriver, base_compressor=compressor)

In [29]:
query = "What is photosynthesis?"

result = compression_retriver.invoke(query)

print(result)

[Document(metadata={}, page_content='Renewable energy sources like solar and wind power are crucial for reducing carbon emissions.')]


In [30]:
for item in result:
    print(item.page_content)

Renewable energy sources like solar and wind power are crucial for reducing carbon emissions.
